In [1]:
import random

import flax
import flax.linen as nn
import jax
import jax.numpy as jnp
import numpy as np
import optax
from flax.training.train_state import TrainState
from stable_baselines3.common.buffers import ReplayBuffer

import sys
sys.path.append('../Environment/Version3_1_2')
from rl_mh import RLMHEnv
from mcmctoolbox.functoolbox import flat

from tqdm.auto import trange

import toml
from types import SimpleNamespace

import json
import pandas as pd
import bridgestan as bs
from posteriordb import PosteriorDatabase

config = toml.load("./rlmcmc/config/ddpg.toml")
args = SimpleNamespace(**config)

# seeding
random.seed(args.seed)
np.random.seed(args.seed)
key = jax.random.PRNGKey(args.seed)
key, actor_key, qf1_key = jax.random.split(key, 3)

import logging
logging.basicConfig(filename="DDPG_single.log", level=logging.INFO)

# Log to Tensorboard
from torch.utils.tensorboard import SummaryWriter
import time

writer = SummaryWriter(f"runs/V3/{str(time.time())}")

In [2]:
# Initialize Agent
class QNetwork(nn.Module):
    """
    Critic Network
    """
    @nn.compact
    def __call__(self, x: jnp.ndarray, a: jnp.ndarray):
        x = jnp.concatenate([x, a], -1)
        x = nn.Dense(48)(x)
        x = nn.softplus(x)
        x = nn.Dense(48)(x)
        x = nn.softplus(x)
        x = nn.Dense(1)(x)
        return x

In [3]:
class Actor(nn.Module):
    sample_dim: int

    @nn.compact
    def __call__(self, obs: jnp.ndarray):
        """
        Mu function
        """
        x, mcmc_noise = jnp.split(obs, [self.sample_dim], axis=1)
        x_star = self.gene_x_star(x, mcmc_noise)

        low_rank_vec_x = self.low_rank_vec(x)
        low_rank_vec_x_star = self.low_rank_vec(x_star)

        return jnp.hstack([low_rank_vec_x, low_rank_vec_x_star])

    def low_rank_vec(self, x):
        """
        Low rank vector and magnification. The shape is the dim of the sample plus a postive scale. i.e. [1.0, 0.0, 0.0, 1.0, 2.0]
        """
        x = nn.Dense(48)(x)
        x = nn.softplus(x)
        x = nn.Dense(48)(x)
        x = nn.softplus(x)
        x = nn.Dense(self.sample_dim, use_bias=True)(x)
        return x

    def cov(self, x):
        """
        Restored low rank vector and magnification to covariance matrix.
        i.e. [1.0, 0.0, 0.0, 1.0, 2.0] -> [[1.0, 0.0], [0.0, 1.0]] + `2.0` * I -> [[3.0, 0.0], [0.0, 3.0]]
        """
        low_rank_vec = self.low_rank_vec(x)

        return (
            low_rank_vec[:, :, None] * low_rank_vec[:, None, :]
        )  # Outer Product

    def gene_x_star(self, x, mcmc_noise):
        cov_x = self.cov(x)
        cov_x_sqrt = jnp.linalg.cholesky(cov_x)
        print(cov_x_sqrt)
        return (
            x
            + jnp.matmul(cov_x_sqrt, mcmc_noise[:, :, None]).squeeze()
        )

    # def cholesky_decompose(self, i, matrices):
    #     updated_matrix = jnp.linalg.cholesky(matrices[i])
    #     return matrices.at[i].set(updated_matrix)

In [4]:
class TrainState(TrainState):
    target_params: flax.core.FrozenDict

In [5]:
# Load DataBase Locally
pdb_path = "/home/congye/Code/PythonProjects/LearningAdaptiveMCMC/Python/posteriordb/posterior_database"
my_pdb = PosteriorDatabase(pdb_path)

# Load Dataset
posterior = my_pdb.posterior("test-SimpleEggBox-test-SimpleEggBox")
stan_code = posterior.model.stan_code_file_path()
# stan_data = json.dumps(posterior.data.values())
stan_data = json.dumps({"sigma": 1, "r": 20})
model = bs.StanModel.from_stan_file(stan_code, stan_data)

# Gold Standard
gs = posterior.reference_draws()
df = pd.DataFrame(gs)
gs_chains = np.zeros((sum(flat(posterior.information['dimensions'].values())),\
                       posterior.reference_draws_info()['diagnostics']['ndraws']))
for i in range(len(df.keys())):
    s = []
    for j in range(len(df[df.keys()[i]])):
        s += df[df.keys()[i]][j]
    gs_chains[i, :] = s
linv = np.linalg.inv(np.cov(gs_chains))

# Extract log-P-pdf and its gradient
log_p = model.log_density


/home/congye/Software/anaconda3/envs/rl/lib/python3.10/site-packages/bridgestan/model.py:102: RuntimeWarning: The version of the compiled model does not match the version of the Python package. Consider recompiling the model.
  warnings.warn(


In [6]:
# Setup env
sample_dim = 2
action_dim = 2 * sample_dim
state_dim = 2 * sample_dim

total_timesteps=100_000
args.total_timesteps = total_timesteps
args.batch_size = 48

env = RLMHEnv(
    log_target_pdf=log_p,
    sample_dim=sample_dim,
    total_timesteps=total_timesteps
    )
rb = ReplayBuffer(
    args.buffer_size,
    env.observation_space,
    env.action_space,
    device='cpu',
    handle_timeout_termination=False
)

In [7]:
# Start
obs, _ = env.reset(seed=args.seed)
actor = Actor(sample_dim=sample_dim)
qf1 = QNetwork()

In [8]:
args.learning_rate = 5e-5

In [9]:
actor_state = TrainState.create(
    apply_fn=actor.apply,
    params=actor.init(actor_key, obs),
    target_params=actor.init(actor_key, obs),
    tx=optax.adam(learning_rate=args.learning_rate),
) # theta

qf1_state = TrainState.create(
    apply_fn=qf1.apply,
    params=qf1.init(qf1_key, obs, env.action_space.sample()),
    target_params=qf1.init(qf1_key, obs, env.action_space.sample()),
    tx=optax.adam(learning_rate=args.learning_rate),
) # q

# actor.apply = jax.jit(actor.apply)
# qf1.apply = jax.jit(qf1.apply)

[[[nan  0.]
  [nan nan]]]
[[[nan  0.]
  [nan nan]]]


In [10]:
@jax.jit
def update_critic(
    actor_state: TrainState,
    qf1_state: TrainState,
    observations: np.ndarray,
    actions: np.ndarray,
    next_observations: np.ndarray,
    rewards: np.ndarray,
    dones: np.ndarray,
):
    # next_state_actions = (actor.apply(actor_state.target_params, next_observations)).clip(-1, 1)
    next_state_actions = (actor.apply(actor_state.target_params, next_observations))
    qf1_next_target = qf1.apply(qf1_state.target_params, next_observations, next_state_actions).reshape(-1)
    next_q_value = (rewards + (1 - dones) * args.gamma * (qf1_next_target)).reshape(-1)

    def mse_loss(params):
        qf1_a_values = qf1.apply(params, observations, actions).squeeze()
        return ((qf1_a_values - next_q_value) ** 2).mean(), qf1_a_values.mean()

    (qf1_loss_value, qf1_a_values), grads = jax.value_and_grad(mse_loss, has_aux=True)(qf1_state.params)
    logging.info("update_critic grads:", grads)

    qf1_state = qf1_state.apply_gradients(grads=grads)
    return qf1_state, qf1_loss_value, qf1_a_values

@jax.jit
def update_actor(
    actor_state: TrainState,
    qf1_state: TrainState,
    observations: np.ndarray,
):
    def actor_loss(params):
        return -qf1.apply(qf1_state.params, observations, actor.apply(params, observations)).mean()

    actor_loss_value, grads = jax.value_and_grad(actor_loss)(actor_state.params)

    logging.info("update_actor grads:", grads)

    actor_state = actor_state.apply_gradients(grads=grads)
    actor_state = actor_state.replace(
        target_params=optax.incremental_update(actor_state.params, actor_state.target_params, args.tau)
    )
    qf1_state = qf1_state.replace(
        target_params=optax.incremental_update(qf1_state.params, qf1_state.target_params, args.tau)
    )
    return actor_state, qf1_state, actor_loss_value

In [11]:
for global_step in trange(args.total_timesteps):
    actions = actor.apply(actor_state.params, obs)

    writer.add_scalars('mcmc/trace_plot', {'x': obs[:, 0].item(), 'y': obs[:, 1].item()}, global_step)

    # actions = (np.array(actions) + np.random.normal(0, args.exploration_noise)).clip(
    #                 env.action_space.low, env.action_space.high
    #             )

    next_obs, rewards, terminateds, truncateds, infos = env.step(np.array(actions, dtype=np.float64))
    real_next_obs = next_obs.copy()
    rb.add(obs, real_next_obs, actions, rewards, terminateds, infos)

    obs = next_obs

    # Training
    if global_step > args.learning_starts:
        data = rb.sample(args.batch_size)
        qf1_state, qf1_loss_value, qf1_a_values = update_critic(
            actor_state=actor_state,
            qf1_state=qf1_state,
            observations=data.observations.reshape(-1, state_dim).numpy(),
            actions=data.actions.numpy(),
            next_observations=data.next_observations.reshape(-1, state_dim).numpy(),
            rewards=data.rewards.flatten().numpy(),
            dones=data.dones.flatten().numpy()
        )
        if global_step % args.policy_frequency == 0:
            actor_state, qf1_state, actor_loss_value = update_actor(
                actor_state=actor_state,
                qf1_state=qf1_state,
                observations=data.observations.reshape(-1, state_dim).numpy(),
            )

        if global_step > args.policy_frequency:
            writer.add_scalar("losses/qf1_loss", qf1_loss_value.item(), global_step)
            writer.add_scalar("losses/qf1_values", qf1_a_values.item(), global_step)
            writer.add_scalar("losses/actor_loss", actor_loss_value.item(), global_step)
            writer.add_scalar("rewards/rewards", rewards.item(), global_step)

  0%|          | 0/100000 [00:00<?, ?it/s]

LinAlgError: SVD did not converge

In [ ]:
state_list = np.array([i for i in env.store_state]).reshape(-1, state_dim)
action_list = np.array([i for i in env.store_action]).reshape(-1, action_dim)

In [ ]:
from matplotlib import pyplot as plt


fig, ax = plt.subplots()
sc = ax.scatter(state_list[1:, 0], state_list[1:, 1], c=(action_list[:, 0]**2 + action_list[:, 1]**2 + action_list[:, 2]**2), cmap='viridis')

plt.colorbar(sc, label='Trace of the Covariance')

In [ ]:
plt.plot(state_list[1:, 0], state_list[1:, 1], 'o-', color='grey', alpha=0.05)

In [ ]:
plt.plot(action_list[:, 0]**2 + action_list[:, 1]**2 + action_list[:, 2]**2)
plt.title("Trace of the Covariance")
plt.show()

In [ ]:
plt.hist(state_list[:, 0], bins=30)
plt.axvline(-20, color="red")
plt.axvline(20, color="red")
plt.show()

In [ ]:
import seaborn as sns

sns.kdeplot(state_list[:, 0])
plt.axvline(-20, color="red")
plt.axvline(20, color="red")
plt.show()

In [ ]:
plt.hist(state_list[:, 1], bins=30)
plt.axvline(-20, color="red")
plt.axvline(20, color="red")
plt.show()

In [ ]:
sns.kdeplot(state_list[:, 1])
plt.axvline(-20, color="red")
plt.axvline(20, color="red")
plt.show()

In [ ]:
plt.plot(env.store_reward)
plt.title("Immediate Reward")
plt.show()

In [ ]:
plt.plot(np.cumsum(env.store_reward))
plt.title("Cumulative Reward")
plt.show()

In [ ]:
plt.plot(state_list[:, 0], label="x", alpha=0.5)
plt.plot(state_list[:, 1], label="y", alpha=0.5)
plt.legend()
plt.show()

In [ ]:
plt.plot(action_list[:, 0]**2 + action_list[:, 2]**2, label="cov1", alpha=0.5)
plt.plot(action_list[:, 0] * action_list[:, 1], label="cov2_3", alpha=0.5)
plt.plot(action_list[:, 1]**2 + action_list[:, 2]**2, label="cov4", alpha=0.5)
plt.legend()
plt.show()

In [ ]:
np.sum(env.store_accetped_status)/len(env.store_accetped_status)

In [ ]:
import pandas as pd
df = pd.DataFrame(
    np.hstack(
        [
    state_list[1:, 0:2],
    np.array(action_list[:, 0]**2 + action_list[:, 2]**2).reshape(-1, 1),
    np.array(action_list[:, 0] * action_list[:, 1]).reshape(-1, 1),
    np.array(action_list[:, 0]**2 + action_list[:, 1]**2).reshape(-1, 1),
    np.array(action_list[:, 1]**2 + action_list[:, 2]**2).reshape(-1, 1),
    np.array(env.store_reward).reshape(-1, 1)[1:],
    np.array(env.store_log_accetance_rate).reshape(-1, 1)[1:],
    np.array(env.store_accetped_status).reshape(-1, 1)[1:]
        ]
    ),
    columns=['x', 'y', 'cov1', 'cov2', 'cov3', 'cov4', 'rewards', 'log_alpha', 'accepted_status']
)
# Find the row with the largest value in the 'rewards' column
max_rewards_row = df.loc[
    [
        df['rewards'].idxmax() - 4,
        df['rewards'].idxmax() - 3,
        df['rewards'].idxmax() - 2,
        df['rewards'].idxmax() - 1,
        df['rewards'].idxmax(),
        df['rewards'].idxmax() + 1,
        df['rewards'].idxmax() + 2,
        df['rewards'].idxmax() + 3,
        df['rewards'].idxmax() + 4
    ]
]

max_rewards_row